In [ ]:
# pip install shap boruta

In [ ]:
# ==========================================================
# Maestría en Ciencia y Análisis de Datos
# Universidad Mayor de San Andrés
# ----------------------------------------------------------
#           Machine Learning y Deep Learning
# ----------------------------------------------------------
#        Rolando Gonzales Martinez, Agosto 2024
# ==========================================================
#                Feature selection
# ==========================================================
import pandas as pd
import shap
import numpy as np
from boruta import BorutaPy
from sklearn.linear_model import ElasticNetCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from lime.lime_tabular import LimeTabularExplainer

In [ ]:
# Cargar el conjunto de datos:
url = 'https://raw.githubusercontent.com/rogon666/UMSA/main/AIMLDL/Datos/enfermedades_cardiacas.csv'

# Cargar los datos en un DataFrame
df= pd.read_csv(url)

# Mostrar las primeras filas del DataFrame
print(df.head())

In [ ]:
# Pre-procesamiento y preparacion:

# Imputar los valores faltantes
df['talasemia'] = df['talasemia'].fillna(df['talasemia'].mean())
df['numerovasos'] = df['numerovasos'].fillna(df['numerovasos'].mean())

# Preparar los datos para el modelo
X = df.drop('diagnostico', axis=1)
y = df['diagnostico'].map({0: 0, 1: 1, 2: 1, 3: 1, 4: 1})

In [ ]:
# Dividir el dataset en conjunto de entrenamiento y prueba
semilla = 666
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2, 
                                                    random_state=semilla)
# Usaremos un modelo RandomForest como estimador benchmark
bosque_aleatorio = RandomForestClassifier(random_state=semilla)

In [ ]:
# 1. Selección de Características con Boruta
# Iniciar Boruta
boruta_selector = BorutaPy(estimator=bosque_aleatorio, 
                           verbose=2, 
                           n_estimators = 'auto',
                           random_state=semilla)

# Ajustar Boruta a los datos de entrenamiento
boruta_selector.fit(X_train.values, y_train.values)

# Características seleccionadas por Boruta
selected_features_boruta = X.columns[boruta_selector.support_].to_list()
print("Características seleccionadas por Boruta:")
print(selected_features_boruta)

In [ ]:
# Explicador SHAP utilizando TreeExplainer (más eficiente para modelos basados en árboles)
bosque_aleatorio.fit(X_train, y_train)
explainer = shap.TreeExplainer(bosque_aleatorio)

# Obtener los valores SHAP
shap_values = explainer.shap_values(X_train)

# Identificar las características más importantes según SHAP
shap_importances = pd.DataFrame(list(zip(X.columns, np.abs(shap_values[1]).mean(axis=0))), columns=['feature', 'shap_importance'])
shap_importances = shap_importances.sort_values(by='shap_importance', ascending=False)
print("Características más importantes según SHAP:")
print(shap_importances)

# Guardar las características seleccionadas por SHAP en una lista
selected_features_shap = shap_importances[shap_importances['shap_importance'] > shap_importances['shap_importance'].mean()]['feature'].to_list()
print("Características seleccionadas por SHAP:")
print(selected_features_shap)

In [ ]:
# 2. Selección de Características con SHAP
# Entrenar un model SVM con todas las características:
from sklearn.svm import SVC

model = SVC(probability=True,
            kernel='linear', random_state=666)
model.fit(X_train, y_train)

# Explicador SHAP utilizando KernelExplainer
explainer = shap.KernelExplainer(model.predict_proba, X_train)

# Obtener los valores SHAP
shap_values = explainer.shap_values(X_train)

# Identificar las características más importantes según SHAP
shap_importances = pd.DataFrame(list(zip(X.columns, np.abs(shap_values[1]).mean(axis=0))), columns=['feature', 'shap_importance'])
shap_importances = shap_importances.sort_values(by='shap_importance', ascending=False)
print("Características más importantes según SHAP:")
print(shap_importances)

# Guardar las características seleccionadas por SHAP en una lista
selected_features_shap = shap_importances[shap_importances['shap_importance'] > shap_importances['shap_importance'].mean()]['feature'].to_list()
print("Características seleccionadas por SHAP:")
print(selected_features_shap)
# sexo: Sexo del individuo, 1 = hombre, 0 = mujer